In [1]:
# introduction to nlp fundamentals in Tensorflow
# NLP has a goal of deriving information out of natural language processing

In [2]:
# nlp also helps in problems which may be sequence to sequence problems

In [3]:
!nvidia-smi -L

/bin/bash: nvidia-smi: command not found


In [4]:
# get the helper functions and import the dataset

In [5]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-07-05 17:37:40--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-07-05 17:37:40 (66.8 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [6]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

In [7]:
# getting a text dataset
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2023-07-05 17:37:45--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.79.128, 108.177.119.128, 108.177.126.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.79.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  1.28MB/s    in 0.5s    

2023-07-05 17:37:46 (1.28 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [8]:
import zipfile
zip_ref = zipfile.ZipFile("nlp_getting_started.zip")
zip_ref.extractall()
zip_ref.close()

In [9]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt


In [10]:
# working on visualising the dataset
train_dataset = pd.read_csv("train.csv")
test_dataset = pd.read_csv("test.csv")


In [11]:
train_dataset.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [12]:
# shuffling the dataset that has been download to avoid learning any unintended patterns

train_df_shuffle = train_dataset.sample(frac = 1, random_state = 42)

In [13]:
train_df_shuffle.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [14]:
len(train_dataset), len(test_dataset)

(7613, 3263)

In [15]:
import random

randomid = random.randint(0,len(train_dataset)-5)
for row in train_df_shuffle[["text","target"]][randomid:randomid+5].itertuples():
  _,text, target = row
  print(text)
  print(target)

[infowars]  Nashville Theater Attack: Will Gun Grabbers Now Demand ÛÏHatchet Control?Û http://t.co/n3yJb8TcPm #nwo
1
Cross-border terrorism: Pakistan caught red-handed again http://t.co/uDj50J3MV4
1
Damn...was wondering where my drone ended up after the freak windstorm...?? https://t.co/dHgGxo7Mcc
1
i just drove with both my parents in the car lmao that was a panic attack waiting to happen
0
Oh and fuck Bill Clinton for bombing us and fuck NATO.
0


In [16]:
from sklearn.model_selection import train_test_split

traindata, valdata, trainlab, vallabel = train_test_split(train_df_shuffle["text"].to_numpy(),
                                                         train_df_shuffle["target" ].to_numpy(),
                                                         random_state = 42,
                                                         test_size = 0.1
                                                         )

In [17]:
# converting the text into numbers

In [18]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
# using the default text vectorisation parameters

text_vectorizer = TextVectorization(max_tokens = 1000000,
                                    standardize = "lower_and_strip_punctuation",
                                    split = "whitespace",
                                    ngrams = None,
                                    output_mode = "int",
                                    output_sequence_length = None,
                                    pad_to_max_tokens = True)



In [19]:
round(sum([len(i.split()) for i in traindata]))/len(traindata)

14.901036345059115

In [20]:
text_vectorizer = TextVectorization(max_tokens = 10000,
                                    output_mode = "int",
                                    output_sequence_length = 15)

In [21]:
# mapping the text vectorization instance to the data give
# fitting the text vectorizer

text_vectorizer.adapt(traindata)
# creating a sample data and vectorizing it

text_vectorizer(["There's a flood in my street"])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [22]:
# choosing the sentence from the training dataset and working on vectorizing it
words_in_vocab = text_vectorizer.get_vocabulary()

In [23]:
words_in_vocab[:5],words_in_vocab[-5:]

(['', '[UNK]', 'the', 'a', 'in'],
 ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1'])

In [24]:
# # using tf embedding layer for our purpose instead of vectroizer
# parameters we care the most about in our embedding layer:
# input_dim = size of directory
# output_dim = size of the output embedding vector
# input_length -  length of the sequences we will be passing to the embedding lyer

In [25]:
from tensorflow.keras import layers
embedding = layers.Embedding(input_dim = 10000,
                             output_dim = 128,
                             input_length = 15)

In [26]:
embedding

In [27]:
random_sentence = random.choice(traindata)
print(f"original text {random_sentence}")
sample_embd = embedding(text_vectorizer([random_sentence]))
print(f"{sample_embd}")

original text Govt plan for Rs40000Cr lifeline to FCI waste of money ask people to store grains fr 3_6_12 months fr emergency enough capacity available nw
[[[-0.03029871  0.02745411 -0.00989367 ... -0.01350371 -0.01167918
    0.0352638 ]
  [ 0.03191246  0.02255486  0.03706516 ...  0.00614228 -0.02989072
    0.0104185 ]
  [-0.0420443  -0.03985264 -0.03011541 ...  0.04264    -0.00677057
    0.02924762]
  ...
  [-0.04297458  0.03539617 -0.03552179 ...  0.0037249   0.00850874
    0.01324758]
  [ 0.0070081  -0.00876833 -0.00484584 ...  0.01980333  0.04635895
    0.03182879]
  [-0.01605855  0.0217711   0.00199706 ...  0.00737829  0.03345377
   -0.04843216]]]


In [28]:
#  we have a way of turning the text sequences
#  into number where we will strat with a baseline and move forward
# model - 0

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model0 = Pipeline([
    ("tfidf",TfidfVectorizer()),
    ("clf",MultinomialNB())
])

model0.fit(traindata,trainlab)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [29]:
# evaluating the baseline model
baseline_score = model0.score(valdata, vallabel)

In [30]:
print(f"Our bbaseline model acieves the accuracy score of {baseline_score * 100}")

Our bbaseline model acieves the accuracy score of 79.26509186351706


In [31]:
baseline_preds = model0.predict(valdata)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [32]:
# building a function to compar eother evaluation metrics for the mdoel
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [33]:
def calculate_results(ytrue, ypred):
  """
  Calculates model precision, recall, fscore and support of a binaary classification model
  """

  model_accuracy = accuracy_score(ytrue, ypred)
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(ytrue, ypred, average = "weighted")
  model_results = {"accuracy":model_accuracy,
                   "precision": model_precision,
                   "recall": model_recall,
                   "f1 score": model_f1}
  return model_results



In [34]:
baseline_scores = calculate_results(ytrue = vallabel,
                                    ypred = baseline_preds)

baseline_scores

{'accuracy': 0.7926509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1 score': 0.7862189758049549}

In [35]:
# creating a feed forward model which will be our second model
# this will be a deep learning dense model trained on the data

# creating a tensorboard callback
from helper_functions import create_tensorboard_callback

save_dir = "model_logs"


In [36]:
from tensorflow.keras import layers
inputs = layers.Input(shape = (1,), dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GlobalAveragePooling1D(name="global_avg_pool_layer")(x)
outputs = layers.Dense(1, activation = "sigmoid")(x)

model1 = tf.keras.Model(inputs, outputs, name = "model1")


In [37]:
model1.summary()

Model: "model1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_avg_pool_layer (Glob  (None, 128)              0         
 alAveragePooling1D)                                             
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
Non-trai

In [38]:
model1.compile(loss = "binary_crossentropy",
               optimizer = tf.keras.optimizers.Adam(),
               metrics = ["accuracy"])



In [39]:
model1_history = model1.fit(x = traindata,
                            y = trainlab,
                            epochs = 5,
                            validation_data = (valdata, vallabel),
                            callbacks = [create_tensorboard_callback(dir_name = save_dir, experiment_name = "model1")])

Saving TensorBoard log files to: model_logs/model1/20230705-173748
Epoch 1/5
215/215 [==============================] - 9s 34ms/step - loss: 0.6112 - accuracy: 0.6891 - val_loss: 0.5350 - val_accuracy: 0.7612
Epoch 2/5
215/215 [==============================] - 8s 39ms/step - loss: 0.4430 - accuracy: 0.8175 - val_loss: 0.4685 - val_accuracy: 0.7887
Epoch 3/5
215/215 [==============================] - 7s 35ms/step - loss: 0.3473 - accuracy: 0.8605 - val_loss: 0.4574 - val_accuracy: 0.7979
Epoch 4/5
215/215 [==============================] - 8s 38ms/step - loss: 0.2848 - accuracy: 0.8921 - val_loss: 0.4653 - val_accuracy: 0.7887
Epoch 5/5
215/215 [==============================] - 7s 32ms/step - loss: 0.2385 - accuracy: 0.9107 - val_loss: 0.4805 - val_accuracy: 0.7861


In [40]:
# visualization of the learned embeddings
# getting the vocabulary for the embeddings frommt he text vectorization layer

words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [41]:
model1.summary()

Model: "model1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_avg_pool_layer (Glob  (None, 128)              0         
 alAveragePooling1D)                                             
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
Non-trai

In [42]:
# n# getting the weight matrix of embedding layer which have been learnt for 5 epochs

embed_weights = model1.get_layer("embedding").get_weights()[0]

In [43]:
embed_weights

array([[-0.01969326,  0.0144259 ,  0.0568055 , ...,  0.00453848,
         0.02802981,  0.04069756],
       [-0.02888773,  0.03292132,  0.01473043, ...,  0.01874198,
         0.02246412, -0.03559449],
       [-0.04980462, -0.00225297,  0.02204776, ...,  0.02872256,
         0.00568779,  0.00271399],
       ...,
       [ 0.02950323, -0.04460489,  0.01423338, ..., -0.04251927,
         0.04242483,  0.01539613],
       [-0.02775231,  0.0043777 ,  0.02455335, ...,  0.05399667,
        -0.00272044,  0.0024637 ],
       [-0.05489343,  0.07246645,  0.04033463, ...,  0.11269627,
        -0.06568453,  0.03735772]], dtype=float32)

In [44]:
embed_weights.shape

(10000, 128)

In [45]:
# visualising te embedding matrix for the tokens created
vocab = text_vectorizer.get_vocabulary()
import io
out_v = io.open('vectors.tsv','w', encoding = 'utf-8')
out_m = io.open("metadata.tsv", 'w', encoding = 'utf-8')

for index, word in enumerate(vocab):
  if index == 0 :
    continue
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")

out_v.close()
out_m.close()

In [46]:
# working with rnn
# the entire premise of the neural netowrk is to use the premise of a previous network to find the representation of sequene data


In [47]:
# working on coding the first instance of RNN
# we will be working with coding the LSTM cell

# creating an LSTM model

from tensorflow.keras import layers

inputs = layers.Input(shape = (1,), dtype = "string")

x = text_vectorizer(inputs)
print(x.shape)
x = embedding(x)
print(x.shape)

x = layers.LSTM(64, return_sequences = True)(x)
print(x.shape)

x = layers.LSTM(64)(x)
print(x.shape)

x = layers.Dense(64, activation = "relu")(x)
outputs = layers.Dense(1, activation = "sigmoid")(x)

model2 = tf.keras.Model(inputs,outputs,name = "model2")

(None, 15)
(None, 15, 128)
(None, 15, 64)
(None, 64)


In [48]:
 model2.compile(loss ="binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ["accuracy"])



In [49]:
model2history = model2.fit(traindata,
                           trainlab,
                           epochs = 5,
                           validation_data = (valdata,vallabel),
                           callbacks = [create_tensorboard_callback(save_dir,
                                                                    "model2")])

Saving TensorBoard log files to: model_logs/model2/20230705-173831
Epoch 1/5
215/215 [==============================] - 17s 54ms/step - loss: 0.2225 - accuracy: 0.9237 - val_loss: 0.5594 - val_accuracy: 0.7848
Epoch 2/5
215/215 [==============================] - 11s 50ms/step - loss: 0.1567 - accuracy: 0.9437 - val_loss: 0.6667 - val_accuracy: 0.7756
Epoch 3/5
215/215 [==============================] - 10s 49ms/step - loss: 0.1340 - accuracy: 0.9485 - val_loss: 0.8864 - val_accuracy: 0.7782
Epoch 4/5
215/215 [==============================] - 10s 48ms/step - loss: 0.1049 - accuracy: 0.9583 - val_loss: 0.7875 - val_accuracy: 0.7848
Epoch 5/5
215/215 [==============================] - 11s 51ms/step - loss: 0.0808 - accuracy: 0.9667 - val_loss: 1.1212 - val_accuracy: 0.7795


In [50]:
# making predictions
model2pred = model2.predict(valdata)
model2pred[:10]

24/24 [==============================] - 1s 10ms/step


array([[3.5242219e-02],
       [4.8479223e-01],
       [9.9999559e-01],
       [4.1045565e-02],
       [1.7640555e-04],
       [9.9999338e-01],
       [9.9267322e-01],
       [9.9999827e-01],
       [9.9999630e-01],
       [3.4812748e-01]], dtype=float32)

In [51]:
model2preds = tf.squeeze(tf.round(model2pred))
model2preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 0., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [52]:
model2result = calculate_results(vallabel,
                                 model2preds)

In [53]:
model2result

{'accuracy': 0.7795275590551181,
 'precision': 0.7849754984084437,
 'recall': 0.7795275590551181,
 'f1 score': 0.7761172470890804}

In [54]:
# we will now be woeking on buidling a gru based rnn
# working on beating th baseline results

# gru standa for gated recurrent network



In [55]:
# another popular and effective rnn component is GRU also called as a gated
# recurent unit which has similar features to an LSTM cell but has lesser number of parameters

# building an rnn using GRU

from tensorflow.keras import layers
inputs = layers.Input(shape = (1,), dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64, activation = "tanh")(x)
# x = layers.GlobalAveragePooling1D()(x)
# x = layers.LSTM(42, return_sequences = True)(x)
# x = layers.GRU(99)(x)
# x = layers.Dense(64, activation = "relu")(x)

output = layers.Dense(1, activation = "sigmoid")(x)

model3 = tf.keras.Model(inputs, output, name = "model3")

In [56]:
model3.summary()

Model: "model3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
__________________________________________________

In [57]:
model3.compile(loss = "binary_crossentropy",
               optimizer = tf.keras.optimizers.Adam(),
               metrics = ["accuracy"])


In [58]:
history3 = model3.fit(traindata,
                      trainlab,
                      epochs = 5,
                      validation_data = (valdata, vallabel),
                      callbacks = [create_tensorboard_callback(save_dir,
                                                               "model3")])

Saving TensorBoard log files to: model_logs/model3/20230705-173932
Epoch 1/5
215/215 [==============================] - 12s 38ms/step - loss: 0.1599 - accuracy: 0.9375 - val_loss: 0.7602 - val_accuracy: 0.7861
Epoch 2/5
215/215 [==============================] - 8s 38ms/step - loss: 0.0840 - accuracy: 0.9680 - val_loss: 0.7426 - val_accuracy: 0.7756
Epoch 3/5
215/215 [==============================] - 10s 47ms/step - loss: 0.0716 - accuracy: 0.9721 - val_loss: 0.9430 - val_accuracy: 0.7690
Epoch 4/5
215/215 [==============================] - 7s 34ms/step - loss: 0.0612 - accuracy: 0.9737 - val_loss: 1.2185 - val_accuracy: 0.7703
Epoch 5/5
215/215 [==============================] - 8s 39ms/step - loss: 0.0522 - accuracy: 0.9771 - val_loss: 1.2965 - val_accuracy: 0.7769


In [59]:
model3preds = tf.squeeze(tf.round(model3.predict(valdata)))
model3preds[:10]

24/24 [==============================] - 1s 5ms/step


<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [60]:
#  calculating the results of model3
model3results = calculate_results(vallabel,model3preds)

In [61]:
# gru and lstm have been found to be pretty similar in terms of performance
# model4 working on a bidirectional rnn
# normally an RNN will go from left to right
# buidling a bidirectional RNN in tf
from tensorflow.keras import layers
inputs = layers.Input(shape = (1,), dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
# x = layers.Bidirectional(layers.GRU(64))(x)
outputs = layers.Dense(1, activation = "sigmoid")(x)
model4 = tf.keras.Model(inputs, outputs, name = "model4")


In [62]:
model4.summary()

Model: "model4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 128)              98816     
 l)                                                              
                                                                 
 dense_4 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,378,945
Non-trai

In [63]:
model4.compile(loss = "binary_crossentropy",
               optimizer = tf.keras.optimizers.Adam(),
               metrics = ["accuracy"])

In [64]:
history4 = model4.fit(traindata,
                      trainlab,
                      epochs = 5,
                      validation_data = (valdata,vallabel),
                      callbacks = [create_tensorboard_callback(save_dir,
                                                               "model4")])

Saving TensorBoard log files to: model_logs/model4/20230705-174020
Epoch 1/5
215/215 [==============================] - 17s 54ms/step - loss: 0.1075 - accuracy: 0.9672 - val_loss: 0.8609 - val_accuracy: 0.7756
Epoch 2/5
215/215 [==============================] - 11s 49ms/step - loss: 0.0512 - accuracy: 0.9781 - val_loss: 1.2472 - val_accuracy: 0.7835
Epoch 3/5
215/215 [==============================] - 11s 50ms/step - loss: 0.0436 - accuracy: 0.9793 - val_loss: 1.2199 - val_accuracy: 0.7743
Epoch 4/5
215/215 [==============================] - 11s 50ms/step - loss: 0.0446 - accuracy: 0.9807 - val_loss: 1.3683 - val_accuracy: 0.7730
Epoch 5/5
215/215 [==============================] - 10s 46ms/step - loss: 0.0460 - accuracy: 0.9785 - val_loss: 1.0309 - val_accuracy: 0.7703


In [65]:
model4preds = tf.squeeze(tf.round(model4.predict(valdata)))

24/24 [==============================] - 1s 8ms/step


In [66]:
model4preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [67]:
model4result = calculate_results(vallabel,
                                 model4preds)

In [68]:
# using CNN for text and sequence based analysis
# we have been using the cnn annd convolutional neural network for images in typically 2D
# but we will be making use of data in 1D

# the process required is still the same.... we will be makin use of embedding and vectorizer
# and afterwards we will be working toeards using the convolutional layers instead of lstm and gru
# based model

In [69]:
# model5: covd1d
embedding_test = embedding(text_vectorizer(["this is a test sentences"]))
conv1d = layers.Conv1D(filters = 32,
                       kernel_size = 5,
                       activation = "relu",
                       padding = "valid")
conv1doutput = conv1d(embedding_test)
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv1doutput)


embedding_test, conv1doutput, max_pool_output

(<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
 array([[[ 0.02040029,  0.033984  , -0.01327113, ...,  0.01470601,
          -0.0387031 ,  0.0302943 ],
         [-0.01410797,  0.0151463 ,  0.01677829, ...,  0.01788941,
          -0.05612107,  0.04505558],
         [ 0.0044648 , -0.03565248,  0.04773413, ...,  0.02465982,
          -0.02015331,  0.02990147],
         ...,
         [ 0.01124273, -0.00401101,  0.05701265, ...,  0.01029284,
           0.00849379,  0.05623604],
         [ 0.01124273, -0.00401101,  0.05701265, ...,  0.01029284,
           0.00849379,  0.05623604],
         [ 0.01124273, -0.00401101,  0.05701265, ...,  0.01029284,
           0.00849379,  0.05623604]]], dtype=float32)>,
 <tf.Tensor: shape=(1, 11, 32), dtype=float32, numpy=
 array([[[1.28877554e-02, 2.88300514e-02, 1.07898312e-02, 7.05598667e-02,
          5.19713201e-03, 5.73279820e-02, 0.00000000e+00, 0.00000000e+00,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 9.38397199e-02,
          3.

In [72]:
# creating the actual model for sequencing
from tensorflow.keras import layers
inputs = layers.Input(shape = (1,), dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(64,kernel_size = 5, activation = "relu", padding = "valid")(x)
x = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(1, activation = "sigmoid")(x)
model5 = tf.keras.Model(inputs,outputs,name = "model5")

In [73]:
model5.compile(loss = "binary_crossentropy",
               optimizer = tf.keras.optimizers.Adam(),
               metrics = ["accuracy"])


In [74]:
history5 = model5.fit(traindata,
                      trainlab,
                      epochs = 5,
                      validation_data = (valdata,vallabel),
                      callbacks = [create_tensorboard_callback(save_dir,
                                                               "model5")])

Saving TensorBoard log files to: model_logs/model5/20230705-174726
Epoch 1/5
215/215 [==============================] - 8s 30ms/step - loss: 0.1260 - accuracy: 0.9604 - val_loss: 0.8784 - val_accuracy: 0.7795
Epoch 2/5
215/215 [==============================] - 5s 23ms/step - loss: 0.0749 - accuracy: 0.9743 - val_loss: 1.0498 - val_accuracy: 0.7743
Epoch 3/5
215/215 [==============================] - 6s 26ms/step - loss: 0.0609 - accuracy: 0.9769 - val_loss: 1.0997 - val_accuracy: 0.7546
Epoch 4/5
215/215 [==============================] - 5s 23ms/step - loss: 0.0556 - accuracy: 0.9759 - val_loss: 1.1775 - val_accuracy: 0.7612
Epoch 5/5
215/215 [==============================] - 5s 23ms/step - loss: 0.0510 - accuracy: 0.9793 - val_loss: 1.1798 - val_accuracy: 0.7559


In [75]:
model5.summary()

Model: "model5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 conv1d_3 (Conv1D)           (None, 11, 64)            41024     
                                                                 
 global_max_pooling1d_2 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_6 (Dense)             (None, 1)                 65   

In [76]:
# making predictions and working ono evaluating the data
model5preds = tf.squeeze(tf.round(model5.predict(valdata)))
model5preds[:10]

24/24 [==============================] - 1s 7ms/step


<tf.Tensor: shape=(10,), dtype=float32, numpy=array([1., 0., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [77]:
model5results = calculate_results(vallabel,
                                  model5preds)

In [78]:
model5results

{'accuracy': 0.7559055118110236,
 'precision': 0.7555301366020079,
 'recall': 0.7559055118110236,
 'f1 score': 0.7550806437203728}

In [79]:
#  making use of transfer learning and pre trained embedding for our purpose
# making use of USE which is the universal entence encoder
import tensorflow_hub as hub

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embed_samples = embed(["When you call the universal sentence encoder on numbers it turns them into numbers"])

In [80]:
embed_samples

<tf.Tensor: shape=(1, 512), dtype=float32, numpy=
array([[ 3.42379957e-02, -9.51867253e-02, -1.18006486e-02,
        -1.59867329e-03, -2.06448939e-02, -5.69342487e-02,
        -1.25469938e-02, -2.67583271e-03, -8.79902318e-02,
         3.62012051e-02, -2.47875117e-02,  5.93638979e-03,
         7.68564781e-03, -1.44476127e-02,  6.87386319e-02,
         4.35329825e-02,  5.59405843e-03, -6.77186996e-02,
        -5.83658107e-02,  6.38084114e-02, -3.36745903e-02,
        -2.12814696e-02,  2.68870424e-02,  6.12838529e-02,
        -7.49350488e-02,  7.35438019e-02, -5.93250878e-02,
        -1.88013678e-03,  1.41302720e-02,  1.74606163e-02,
         3.24174501e-02,  1.34069631e-02, -3.18934880e-02,
         1.64720230e-02, -8.62289742e-02,  2.89589036e-02,
        -2.78575625e-02,  3.77287380e-02, -1.37369232e-02,
         1.81296431e-02,  1.20376740e-02,  2.35112086e-02,
         4.95251678e-02, -6.18886342e-03,  5.15174158e-02,
         7.87481386e-03,  5.33567928e-02, -2.01496240e-02,
      

In [81]:
sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape = [],
                                        dtype = tf.string,
                                        trainable = False,
                                        name = "USE")

In [92]:
# creating aa model using the sequential api
model6 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(64, activation = "relu"),
    layers.Dense(1, activation = "sigmoid")
])

In [93]:
model6.compile(loss = "binary_crossentropy",
               optimizer = tf.keras.optimizers.Adam(),
               metrics = ["accuracy"])

In [94]:
history6 = model6.fit(traindata,
                          trainlab,
                          epochs = 5,
                          validation_data = (valdata, vallabel),
                          callbacks = [create_tensorboard_callback(save_dir,"model6")])

Saving TensorBoard log files to: model_logs/model6/20230705-185002
Epoch 1/5
215/215 [==============================] - 5s 18ms/step - loss: 0.5073 - accuracy: 0.7792 - val_loss: 0.4493 - val_accuracy: 0.8058
Epoch 2/5
215/215 [==============================] - 4s 19ms/step - loss: 0.4149 - accuracy: 0.8154 - val_loss: 0.4412 - val_accuracy: 0.8097
Epoch 3/5
215/215 [==============================] - 3s 14ms/step - loss: 0.4017 - accuracy: 0.8205 - val_loss: 0.4377 - val_accuracy: 0.8084
Epoch 4/5
215/215 [==============================] - 3s 15ms/step - loss: 0.3928 - accuracy: 0.8281 - val_loss: 0.4309 - val_accuracy: 0.8110
Epoch 5/5
215/215 [==============================] - 3s 14ms/step - loss: 0.3862 - accuracy: 0.8298 - val_loss: 0.4275 - val_accuracy: 0.8202


In [95]:
# making predictions using the trained model
model6pred = tf.squeeze(tf.round(model6.predict(valdata)))

24/24 [==============================] - 1s 11ms/step


In [96]:
model6pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [97]:
model6result = calculate_results(vallabel,
                                 model6pred)

In [98]:
model6result

{'accuracy': 0.8202099737532809,
 'precision': 0.8210875591779185,
 'recall': 0.8202099737532809,
 'f1 score': 0.8192635530876636}

In [100]:
# uploading and analysing the model log using tensorboard dev
!tensorboard dev upload --logdir ./model_logs/ \
--name "NLP modelling experiments course" \
--description "comparing the performance of all the models we have trained so far" \
--one_shot

2023-07-05 19:26:43.823055: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

***** TensorBoard Uploader *****

This will upload your TensorBoard logs to https://tensorboard.dev/ from
the following directory:

./model_logs/

This TensorBoard will be visible to everyone. Do not upload sensitive
data.

Your use of this service is subject to Google's Terms of Service
<https://policies.google.com/terms> and Privacy Policy
<https://policies.google.com/privacy>, and TensorBoard.dev's Terms of Service
<https://tensorboard.dev/policy/terms/>.

This notice will not be shown again while you are logged into the uploader.
To log out, run `tensorboard dev auth revoke`.

Continue? (yes/NO) yes

To sign in with the TensorBoard uploader:

1. On your computer or phone, visit:

   https://www.google.com/device

2. Sign in with your Google account, then enter:

   VVN-CYD-NGG



New experiment created. View your TensorBoard at: https://tensorboard.dev/experi

In [101]:
# deleting an experiment
!tensorboard dev list

2023-07-05 19:30:16.658321: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
https://tensorboard.dev/experiment/WSqZ79DKRiOw1EQl7qXFwQ/
	Name                 NLP modelling experiments course
	Description          comparing the performance of all the models we have trained so far
	Id                   WSqZ79DKRiOw1EQl7qXFwQ
	Created              2023-07-05 19:27:44 (2 minutes ago)
	Updated              2023-07-05 19:27:53 (2 minutes ago)
	Runs                 14
	Tags                 5
	Scalars              210
	Tensor bytes         0
	Binary object bytes  3093285
Total: 1 experiment(s)


In [102]:
!tensorboard dev delete --experiment_id WSqZ79DKRiOw1EQl7qXFwQ

2023-07-05 19:31:04.513321: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Deleted experiment WSqZ79DKRiOw1EQl7qXFwQ.
